# Churn Prediction with Machine Learning

## About Dataset

**CLIENTNUM** - Client number. Unique identifier for the customer holding the account. <br>
**Attrition_Flag** - Internal event (customer activity) variable - if the account is closed then 1 else 0. <br>

*Attrited customers are the group of customers that has been lost through time for any reason. The process of losing a client is referred to as customer attrition, also known as customer churn.*<br>

**Customer_Age** -  Customer's age in years <br>
**Gender** - Customer's gender. - M=Male, F=Female <br>
**Dependent_count** -  Number of dependents <br>
**Education_Level** - Educational Qualification of the account holder (example: high school, college graduate, etc.). <br>
**Marital_Status** - Married, Single, Divorced, Unknown <br>
**Income_Category** - Annual Income Category of the account holder (< $40K, $40K - 60K, $60K - $80K, $80K-$120K, >) <br>
**Card_Category** -  Type of Card (Blue, Silver, Gold, Platinum) <br>
**Months_on_book** - Period of relationship with bank <br>
**Total_Relationship_Count** - Total number of products held by the customer. <br>
**Months_Inactive_12_mon** - Number of months inactive in the last 12 months <br>
**Contacts_Count_12_mon** - Number of contacts in the last 12 months <br>
**Credit_Limit** - Credit limit on the credit card <br>
**Total_Revolving_Bal** - Total revolving balance on the credit card<br>
**Avg_Open_To_Buy** - Open to buy credit line (average of last 12 months) <br>
**Total_Amt_Chng_Q4_Q1** - Change in transaction amount (Q4 over Q1) <br>
**Total_Trans_Amt** - Total transaction amount (Last 12 months) <br>
**Total_Trans_Ct** - Total transaction count (Last 12 months) <br>
**Total_Ct_Chng_Q4_Q1** - Change in transaction count (Q4 over Q1) <br>
**Avg_Utilization_Ratio** - Average card utilization ratio <br>

In [20]:
# libraries

import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import missingno as msno
from datetime import date
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.neighbors import LocalOutlierFactor
from sklearn.preprocessing import MinMaxScaler, LabelEncoder,StandardScaler, RobustScaler

pd.set_option("display.max_columns",None)
pd.set_option("display.width",500)
sns.set(rc={"figure.figsize":(12,12)})

In [21]:
data = pd.read_csv('./data/BankChurners.csv')
data = data.drop(['Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1',
                  'Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2'],
                axis='columns')
data.head()

,CLIENTNUM,Attrition_Flag,Customer_Age,Gender,Dependent_count,Education_Level,Marital_Status,Income_Category,Card_Category,Months_on_book,Total_Relationship_Count,Months_Inactive_12_mon,Contacts_Count_12_mon,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio
0,768805383,Existing Customer,45,M,3,High School,Married,$60K - $80K,Blue,39,5,1,3,12691.0,777,11914.0,1.335,1144,42,1.625,0.061
1,818770008,Existing Customer,49,F,5,Graduate,Single,Less than $40K,Blue,44,6,1,2,8256.0,864,7392.0,1.541,1291,33,3.714,0.105
2,713982108,Existing Customer,51,M,3,Graduate,Married,$80K - $120K,Blue,36,4,1,0,3418.0,0,3418.0,2.594,1887,20,2.333,0.000
3,769911858,Existing Customer,40,F,4,High School,Unknown,Less than $40K,Blue,34,3,4,1,3313.0,2517,796.0,1.405,1171,20,2.333,0.760
4,709106358,Existing Customer,40,M,3,Uneducated,Married,$60K - $80K,Blue,21,5,1,0,4716.0,0,4716.0,2.175,816,28,2.500,0.000


In [22]:
data.shape

(10127, 21)

In [23]:
data.isna().sum()

CLIENTNUM                   0
Attrition_Flag              0
Customer_Age                0
Gender                      0
Dependent_count             0
Education_Level             0
Marital_Status              0
Income_Category             0
Card_Category               0
Months_on_book              0
Total_Relationship_Count    0
Months_Inactive_12_mon      0
Contacts_Count_12_mon       0
Credit_Limit                0
Total_Revolving_Bal         0
Avg_Open_To_Buy             0
Total_Amt_Chng_Q4_Q1        0
Total_Trans_Amt             0
Total_Trans_Ct              0
Total_Ct_Chng_Q4_Q1         0
Avg_Utilization_Ratio       0
dtype: int64

In [24]:
print(data["CLIENTNUM"].nunique())
print(len(data))

# All the client numbers are unique, no duplicate

10127
10127


In [25]:
data["Total_Trans_Amt"].max()

18484

In [26]:
data.groupby("CLIENTNUM").aggregate({"Total_Trans_Amt":"sum"}).sort_values(by='Total_Trans_Amt',ascending=False).head()

,Total_Trans_Amt
CLIENTNUM,
718140783,18484
717642633,17995
801036033,17744
716004258,17634
713758758,17628


In [27]:
print(data["Months_Inactive_12_mon"].nunique())
print(data["Months_Inactive_12_mon"].unique())

print(data["Contacts_Count_12_mon"].nunique())
print(data["Contacts_Count_12_mon"].unique())

7
[1 4 2 3 6 0 5]
7
[3 2 0 1 4 5 6]


# Customer Segmentation with RFM

In [28]:
afm = pd.DataFrame()
afm["client_id"]= data["CLIENTNUM"]
afm["activity"] = (12 - data["Months_Inactive_12_mon"])
afm["frequency"] = data["Contacts_Count_12_mon"]
afm["monetary"] = data["Total_Trans_Amt"]
afm.head()

,client_id,activity,frequency,monetary
0,768805383,11,3,1144
1,818770008,11,2,1291
2,713982108,11,0,1887
3,769911858,8,1,1171
4,709106358,11,0,816


In [29]:
afm.groupby("client_id").agg("sum").head()

,activity,frequency,monetary
client_id,,,
708082083,9,3,15149
708083283,11,3,992
708084558,9,3,1447
708085458,10,2,3940
708086958,7,2,4369


In [30]:
afm["activity"].unique()

array([11,  8, 10,  9,  6, 12,  7], dtype=int64)

In [31]:
afm["activity_score"] = afm.apply(lambda _: ' ', axis=1)
afm.head()

,client_id,activity,frequency,monetary,activity_score
0,768805383,11,3,1144,
1,818770008,11,2,1291,
2,713982108,11,0,1887,
3,769911858,8,1,1171,
4,709106358,11,0,816,


In [32]:
afm.loc[afm["activity"] == 12, "activity_score"] = 5
afm.loc[afm["activity"] == 11, "activity_score"] = 4
afm.loc[afm["activity"] == 10, "activity_score"] = 4
afm.loc[afm["activity"] == 9, "activity_score"] = 3
afm.loc[afm["activity"] == 8, "activity_score"] = 3
afm.loc[afm["activity"] == 7, "activity_score"] = 2
afm.loc[afm["activity"] == 6, "activity_score"] = 1

In [34]:
afm.head(10)

,client_id,activity,frequency,monetary,activity_score
0,768805383,11,3,1144,4
1,818770008,11,2,1291,4
2,713982108,11,0,1887,4
3,769911858,8,1,1171,3
4,709106358,11,0,816,4
5,713061558,11,2,1088,4
6,810347208,11,3,1330,4
7,818906208,10,2,1538,4
8,710930508,10,0,1350,4
9,719661558,9,3,1441,3


In [35]:
afm["monetary"].unique()

array([ 1144,  1291,  1887, ..., 10291,  8395, 10294], dtype=int64)

In [36]:
afm["monetary_score"] = pd.qcut(afm["monetary"], 5, labels=[1,2,3,4,5])

In [37]:
afm.sort_values(by="monetary",ascending=False).head()

,client_id,activity,frequency,monetary,activity_score,monetary_score
9964,718140783,7,2,18484,2,5
10073,717642633,9,3,17995,3,5
10097,801036033,10,3,17744,4,5
9601,716004258,11,3,17634,4,5
9341,713758758,10,2,17628,4,5


In [38]:
afm["frequency_score"] = pd.qcut(afm["frequency"].rank(method="first"), 5, labels=[1,2,3,4,5])

In [42]:
afm["AFM_score"] = (afm["frequency_score"].astype(str) + afm["activity_score"].astype(str))

In [44]:
# champion users
afm[afm["AFM_score"] == "55"] # only 7 users

,client_id,activity,frequency,monetary,activity_score,monetary_score,frequency_score,AFM_score
1721,711509658,12,4,1088,5,1,5,55
2126,715517733,12,4,936,5,1,5,55
2671,827984658,12,4,1885,5,1,5,55
4302,715405758,12,4,2383,5,2,5,55
5251,826077033,12,4,2423,5,2,5,55
8442,796083783,12,6,2278,5,2,5,55
9633,718297683,12,3,7597,5,5,5,55


In [46]:
# AFM naming

seg ={
    r'[1-2][1-2]': 'hibernating',
    r'[1-2][3-4]': 'at risk',
    r'[1-2]5': 'cant_loose',
    r'3[1-2]': 'about_to_sleep',
    r'33': 'need_attention',
    r'[3-4][4-5]': 'loyal_customers',
    r'41': 'promising',
    r'51': 'new_customers',
    r'[4-5][2-3]': 'potential_loyalists',
    r'5[4-5]': 'champions',
}

In [47]:
afm["segment"] = afm["AFM_score"].replace(seg,regex=True)

In [48]:
afm.head()

,client_id,activity,frequency,monetary,activity_score,monetary_score,frequency_score,AFM_score,segment
0,768805383,11,3,1144,4,1,3,34,loyal_customers
1,818770008,11,2,1291,4,1,1,14,at risk
2,713982108,11,0,1887,4,1,1,14,at risk
3,769911858,8,1,1171,3,1,1,13,at risk
4,709106358,11,0,816,4,1,1,14,at risk


In [51]:
afm[["segment","frequency","activity","monetary"]].groupby("segment").agg(["mean","count"])

frequency         activity           monetary      
                         mean count       mean count         mean count
segment                                                                
about_to_sleep       2.421053    57   6.666667    57  4227.263158    57
at risk              1.436113  3921   9.787554  3921  3951.443254  3921
cant_loose           1.250000    12  12.000000    12  3528.500000    12
champions            3.912963  1080  10.386111  1080  5313.365741  1080
hibernating          1.347458   118   6.576271   118  4370.483051   118
loyal_customers      2.738270  2174  10.412603  2174  4699.225851  2174
need_attention       2.461802   877   8.881414   877  4641.391106   877
new_customers        4.040000    25   6.000000    25  5314.040000    25
potential_loyalists  3.487725  1833   8.818876  1833  4378.907801  1833
promising            3.000000    30   6.000000    30  4103.966667    30

In [52]:
afm.to_csv("afm.csv")